In [2]:
import dspy

mini = dspy.OpenAI(model = "gpt-4o-mini")
colbertv2_wiki17_abstracts = dspy.ColBERTv2(url = 'http://20.102.90.50:2017/wiki17_abstracts')

dspy.settings.configure(lm = mini, rm = colbertv2_wiki17_abstracts)

/Users/kevinvegda/Desktop/code/dspy-projects/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/kevinvegda/Desktop/code/dspy-projects/.venv/lib/python3.12/site-packages/dsp/templates/template_v2.py:35: SyntaxWarning: invalid escape sequence '\s'
  match = re.search("(.*)(\s){(.*)}\s(.*\${.*})", template)
/Users/kevinvegda/Desktop/code/dspy-projects/.venv/lib/python3.12/site-packages/dsp/templates/template_v2.py:42: SyntaxWarning: invalid escape sequence '\s'
  match = re.search("(.*)(\s){(.*)}", template)


In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from dspy.datasets import HotPotQA

dataset = HotPotQA(train_seed = 1, train_size = 20, eval_seed = 42, dev_size = 50, test_size = 0)
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

len(trainset), len(devset)

Generating test split: 100%|██████████| 7405/7405 [00:01<00:00, 7222.01 examples/s]
/Users/kevinvegda/Desktop/code/dspy-projects/.venv/lib/python3.12/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


(20, 50)

In [6]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers"""

    context = dspy.InputField(desc = "may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc = "often between 1 and 5 words")

In [7]:
class RAG(dspy.Module):
    def __init__(self, num_passages = 3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k = num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context = context, question = question)
        return dspy.Prediction(context = context, answer = prediction.answer)

In [8]:
import dspy.evaluate
from dspy.teleprompt import BootstrapFewShot

def validate_context_and_answer(example, pred, trace = None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

optimizer = BootstrapFewShot(metric=validate_context_and_answer)
compiled_rag = optimizer.compile(RAG(), trainset = trainset)

 55%|█████▌    | 11/20 [00:13<00:10,  1.21s/it]

Bootstrapped 4 full traces after 12 examples in round 0.


In [9]:
my_question = "How many floors does the Empire State Building have?"

pred = compiled_rag(my_question)

print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

Question: How many floors does the Empire State Building have?
Predicted Answer: 102 floors
Retrieved Contexts (truncated): ['Empire State Building | The Empire State Building is a 102-story skyscraper located on Fifth Avenue between West 33rd and 34th Streets in Midtown, Manhattan, New York City. It has a roof height of 1,2...', 'Empire, Colombo | The Empire is a residential complex consisting of two identical towers of 33 and 37 floors. The towers consists of a total of 104 apartments, including four penthouses with floor spa...', '15 Penn Plaza | 15 Penn Plaza, also known as the Vornado Tower, is a proposed 68-story tower in the Midtown Manhattan section of New York City planned by Vornado Realty Trust. It would have 430 units ...']


In [11]:
mini.inspect_history(n=1)





Answer questions with short factoid answers

---

Question: At My Window was released by which American singer-songwriter?
Answer: John Townes Van Zandt

Question: "Everything Has Changed" is a song from an album released under which record label ?
Answer: Big Machine Records

Question: The Victorians - Their Story In Pictures is a documentary series written by an author born in what year?
Answer: 1950

Question: Which Pakistani cricket umpire who won 3 consecutive ICC umpire of the year awards in 2009, 2010, and 2011 will be in the ICC World Twenty20?
Answer: Aleem Sarwar Dar

Question: Having the combination of excellent foot speed and bat speed helped Eric Davis, create what kind of outfield for the Los Angeles Dodgers?
Answer: "Outfield of Dreams"

Question: Who is older, Aleksandr Danilovich Aleksandrov or Anatoly Fomenko?
Answer: Aleksandr Danilovich Aleksandrov

Question: The Organisation that allows a community to influence their operation or use and to enjoy the benefits a

In [12]:
for name, parameter in compiled_rag.named_predictors():
    print(name)
    print(parameter.demos[0])
    print()

generate_answer
Example({'augmented': True, 'context': ['Tae Kwon Do Times | Tae Kwon Do Times is a magazine devoted to the martial art of taekwondo, and is published in the United States of America. While the title suggests that it focuses on taekwondo exclusively, the magazine also covers other Korean martial arts. "Tae Kwon Do Times" has published articles by a wide range of authors, including He-Young Kimm, Thomas Kurz, Scott Shaw, and Mark Van Schuyver.', "Kwon Tae-man | Kwon Tae-man (born 1941) was an early Korean hapkido practitioner and a pioneer of the art, first in Korea and then in the United States. He formed one of the earliest dojang's for hapkido in the United States in Torrance, California, and has been featured in many magazine articles promoting the art.", 'Hee Il Cho | Cho Hee Il (born October 13, 1940) is a prominent Korean-American master of taekwondo, holding the rank of 9th "dan" in the martial art. He has written 11 martial art books, produced 70 martial art tra

In [14]:
from dspy.evaluate.evaluate import Evaluate

evaluate_on_hotpotqa = Evaluate(devset = devset, num_threads=1, display_progress=False, display_table = 5)

metric = dspy.evaluate.answer_exact_match
evaluate_on_hotpotqa(compiled_rag, metric)

Average Metric: 23 / 50  (46.0%)


/Users/kevinvegda/Desktop/code/dspy-projects/.venv/lib/python3.12/site-packages/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['False' '✔️ [True]' '✔️ [True]' '✔️ [True]' '✔️ [True]']' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,example_answer,gold_titles,context,pred_answer,answer_exact_match
0,What is T. J. Lottie's husband's nickname?,Rip,"{'Richard Hamilton (basketball)', 'T. J. Lottie'}","['T. J. Lottie | T. J. Lottie is an American singer. She was a member of the R&B group So Plush, which was on the...",Not provided,False
1,Aniki-Bóbó is akin to a children's counting rhyme that has existed in various forms since well before when?,1820,"{'Aniki-Bóbó', 'Eeny, meeny, miny, moe'}",['Aniki-Bóbó | Aniki-Bóbó is a 1942 Portuguese film directed by Manoel de Oliveira. It is his first feature-length film. The actors are mostly children from...,1820,✔️ [True]
2,Estonian Philharmonic Chamber Choir won the grammy Award for Best Choral Performance for two songs by a composer born in what year ?,1935,"{'Estonian Philharmonic Chamber Choir', 'Arvo Pärt'}",['Estonian Philharmonic Chamber Choir | The Estonian Philharmonic Chamber Choir (EPCC) is a professional choir based in Estonia. It was founded in 1981 by Tõnu...,1935,✔️ [True]
3,"What is the nationality of the woman who performed ""A Message to Your Heart"" at the Eurovision Song Contest in 1991?",British,"{'A Message to Your Heart', 'Samantha Womack'}","['A Message to Your Heart | ""A Message to Your Heart"", written and composed by Paul Curtis, was the United Kingdom\'s entry at the Eurovision...",British,✔️ [True]
4,What musical was based on the 2003 film directed by Jon Favreau?,Elf,"{'Elf: The Musical', 'Elf (film)'}",['The Big Empty (2003 film) | The Big Empty is a 2003 drama film directed and written by Steve Anderson. It stars Jon Favreau as...,Elf,✔️ [True]


46.0

In [15]:
def gold_passages_retrieved(example, pred, trace = None):
    gold_titles = set(map(dspy.evaluate.normalize_text, example['gold_titles']))
    found_titles = set(map(dspy.evaluate.normalize_text, [c.split('|')[0] for c in pred.context]))

    return gold_titles.issubset(found_titles)

compiled_rag_retrieval_score = evaluate_on_hotpotqa(compiled_rag, metric=gold_passages_retrieved)

Average Metric: 13 / 50  (26.0%)


/Users/kevinvegda/Desktop/code/dspy-projects/.venv/lib/python3.12/site-packages/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['False' '✔️ [True]' 'False' 'False' 'False']' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,example_answer,gold_titles,context,pred_answer,gold_passages_retrieved
0,What is T. J. Lottie's husband's nickname?,Rip,"{'Richard Hamilton (basketball)', 'T. J. Lottie'}","['T. J. Lottie | T. J. Lottie is an American singer. She was a member of the R&B group So Plush, which was on the...",Not provided,False
1,Aniki-Bóbó is akin to a children's counting rhyme that has existed in various forms since well before when?,1820,"{'Aniki-Bóbó', 'Eeny, meeny, miny, moe'}",['Aniki-Bóbó | Aniki-Bóbó is a 1942 Portuguese film directed by Manoel de Oliveira. It is his first feature-length film. The actors are mostly children from...,1820,✔️ [True]
2,Estonian Philharmonic Chamber Choir won the grammy Award for Best Choral Performance for two songs by a composer born in what year ?,1935,"{'Estonian Philharmonic Chamber Choir', 'Arvo Pärt'}",['Estonian Philharmonic Chamber Choir | The Estonian Philharmonic Chamber Choir (EPCC) is a professional choir based in Estonia. It was founded in 1981 by Tõnu...,1935,False
3,"What is the nationality of the woman who performed ""A Message to Your Heart"" at the Eurovision Song Contest in 1991?",British,"{'A Message to Your Heart', 'Samantha Womack'}","['A Message to Your Heart | ""A Message to Your Heart"", written and composed by Paul Curtis, was the United Kingdom\'s entry at the Eurovision...",British,False
4,What musical was based on the 2003 film directed by Jon Favreau?,Elf,"{'Elf: The Musical', 'Elf (film)'}",['The Big Empty (2003 film) | The Big Empty is a 2003 drama film directed and written by Steve Anderson. It stars Jon Favreau as...,Elf,False
